![](images/Header.png)

# Guerra de consolas - Visualización de datos - PEC 3
### Por Alejandro Pérez Manrique - Mayo 2022

![figure1](images/videogame_consoles.jpg)

****

Uno de los temas más extremadamente debatidos es "¿Qué consola de videojuegos es mejor?" o "¿Qué empresa de videojuegos es mejor?". Basta con entrar en Twitter y comenzar a indagar que veremos como muchos _"sonyer"_ defenderán a _Playstation_ ante los _xboxers_, como los jugadores de PC siempre alardean de su supremacía ante los gráficos y su gran variedad de juegos buenos y baratos, como los _nintenderos_ justifican que ellos son los que verdaderamente se merecen el podio a lo que se considera un gran "jugón".

Pero al final, lo que vale en este mundo son los números, es por ello que gracias a la página [vgchartz](https://www.vgchartz.com) podemos obtener un _dataset_ sobre ventas de videojuegos por plataformas. En [Kaggle](https://www.kaggle.com/datasets/gregorut/videogamesale) podemos encontrar el conjunto de datos con el que vamos a trabajar. En dicho _dataset_ se refleja las ventas de videojuegos, pero a lo que nosotros nos va a interesar son las ventas globales. Es hora de ver que plataforma de videojuegos es la que más juegos ha vendido, pero sobre todo, veremos también que compañía se alza con la victoria.

> **NOTA**: PC no es una compañía como tal, así que la vamos a considerar como una aunque la realidad nos diga lo contrario dado que hay muchos fabricantes que ensamblan ordenadores.

## Guerra de consolas - Code
****

In [17]:
# Importamos librerías
import pandas as pd
import numpy as np
import csv
import arrow

In [18]:
# Lectura del fichero csv
df = pd.read_csv('Videogames/vgsales.csv')

# Visualizamos las primeras columnas
df.head(5)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [19]:
# Información sobre el dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


Parece que tenemos valores nulos en cuanto al año de publicación. No nos quedará más remedio que eliminar esos registros ya que desconocemos la fecha de publicación y el hecho de realizar la búsqueda de los años puede ser muy laborioso.

In [20]:
# Eliminación de nulos.
df = df[df['Year'].notna()]

### Ventas globales
****
#### Por plataforma de videojuegos
****
Comenzaremos por las ventas globales ya que nos darán la información sobre el éxito global de la consola. Luego indagaremos por países.

In [21]:
# Ordenamos por año
df = df.sort_values('Year')

# Convertimos columna año a formato fecha
df['Year'] = pd.to_datetime(df['Year'], format="%Y")

# Nos quedamos con las columnas necesarias
df_global_war = df.loc[:, ["Platform", "Year", "Global_Sales"]]

# Exportamos nuestro nuevo csv
df_global_war.to_csv("Videogames/global_war_consoles.csv",
                     encoding="utf-8")

In [22]:
def process_csv(filename):
    '''The function aims to obtain a dictionary of
    videoconsoles and their global videogame sales.
    
    The function has as input a .csv file containing the
    years, the name of the videoconsole and their videogame sales.
    Launch a dictionary separating by platforms, the years they
    have appeared and the total videogame sales in this year.
    
    Argument:
    - filename: csv file with data
    
    Return:
    - platforms: Dictionary with sorted data
    '''
    platforms = {}
    dates = []
    
    with open(filename, 'rt', encoding='utf-8') as f:
        csv_reader = csv.DictReader(f)
        for row in csv_reader:
            platform = row['Platform']
            sales = row['Global_Sales']
            year = arrow.get(row['Year'], 'YYYY').format('YYYY')
            date = f'{year}'
            
            if platform not in platforms:
                platforms[platform] = {}
                    
            if date not in platforms[platform]:
                platforms[platform][date] = float(sales)
                    
            else:
                platforms[platform][date] += float(sales)
                
            if date not in dates:
                dates.insert(0, date)
                    
    return platforms

In [23]:
# Usamos la función
df_global_war = process_csv('Videogames/global_war_consoles.csv')

# Convertimos el diccionario en un dataframe
df_global = pd.DataFrame(df_global_war)

# Transponemos columnas por filas
df_global = df_global.transpose()

# Comprobamos orden de las columnas
df_global.columns

Index(['1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2020', '2017'],
      dtype='object')

Vemos que por algún motivo la últimas dos columnas no están en orden, no es un problema grave, vamos a cambiarles de lugar.

In [24]:
# Corregimos orden de las columnas
last_column = df_global.pop('2020') 
  
# Insertamos de nuevo la columna
df_global.insert(38, '2020', last_column)

# observamos de nuevo el resultado
df_global.columns

Index(['1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2020'],
      dtype='object')

El siguiente paso será convertir los valores _string_ a números y realizar la suma acumulada por años.

In [25]:
# Damos valores a los na
df_global.fillna(0, inplace=True)

# Change data type from str to int
df_global = df_global[:].astype('float')

# Realizamos sumas acumuladas
for i in range(len(df_global.columns)):
    if i == 0:
        pass
    else:
        df_global[df_global.columns[i]] = df_global[
            df_global.columns[i]] + df_global[df_global.columns[i-1]]

El dataset está preparado para comenzar a elaborar una gráfica dinámica con librerías de Python o enviarlo a _Flourish_.

In [26]:
# Exportación del dataset listo
df_global.to_csv("Videogames/global_war_sales.csv",
                     encoding="utf-8")

****
#### Por empresa
****

Ahora vamos a añadir algo más de información a nuestro dataset de forma que tengamos las empresas de cada consola y veamos quien tiene la supremacía.

Para ello, vamos a añadir una nueva columna en la que vamos a generalizar la empresa de cada una de las plataformas.

In [27]:
# Observamos que plataformas tenemos
df.Platform.unique()

array(['2600', 'NES', 'PC', 'DS', 'GB', 'SNES', 'GEN', 'GG', 'NG', 'SCD',
       'SAT', 'PS', '3DO', 'TG16', 'N64', 'PCFX', 'DC', 'WS', 'PS2',
       'GBA', 'XB', 'GC', 'PSP', 'X360', 'PS3', 'Wii', '3DS', 'PSV',
       'WiiU', 'PS4', 'XOne'], dtype=object)

Para saber que plataforma pertenece a cada empresa habrá que realizar una búsqueda en _Google_, pero para evitar este trabajo vamos a realizar un resumen:

- Nintendo: NES, DS, GB, SNES, N64, GBA, GC, Wii, 3DS, WiiU
- Playstation: PS, PS2, PSP, PS3, PSV, PS4
- Microsoft: XB, X360, XOne
- Atari: 2600
- SEGA: GEN, GG, SCD, SAT, DC
- NEOGEO: NG
- Panasonic: 3DO
- NEC: TG6, PCFX, WS
- PC: PC

Teniendo esto claro vamos a tratar de añadir esta información.

In [36]:
# Añadimos información de las empresas
df_completed = df.copy(deep=True)
df_completed['Brand'] = None

df_completed.loc[df_completed.Platform == 'NES', 'Brand'] = 'Nintendo'
df_completed.loc[df_completed.Platform == 'DS', 'Brand'] = 'Nintendo'
df_completed.loc[df_completed.Platform == 'GB', 'Brand'] = 'Nintendo'
df_completed.loc[df_completed.Platform == 'SNES', 'Brand'] = 'Nintendo'
df_completed.loc[df_completed.Platform == 'N64', 'Brand'] = 'Nintendo'
df_completed.loc[df_completed.Platform == 'GBA', 'Brand'] = 'Nintendo'
df_completed.loc[df_completed.Platform == 'GC', 'Brand'] = 'Nintendo'
df_completed.loc[df_completed.Platform == 'Wii', 'Brand'] = 'Nintendo'
df_completed.loc[df_completed.Platform == '3DS', 'Brand'] = 'Nintendo'
df_completed.loc[df_completed.Platform == 'WiiU', 'Brand'] = 'Nintendo'
df_completed.loc[df_completed.Platform == 'PS', 'Brand'] = 'Playstation'
df_completed.loc[df_completed.Platform == 'PS2', 'Brand'] = 'Playstation'
df_completed.loc[df_completed.Platform == 'PSP', 'Brand'] = 'Playstation'
df_completed.loc[df_completed.Platform == 'PS3', 'Brand'] = 'Playstation'
df_completed.loc[df_completed.Platform == 'PSV', 'Brand'] = 'Playstation'
df_completed.loc[df_completed.Platform == 'PS4', 'Brand'] = 'Playstation'
df_completed.loc[df_completed.Platform == '2600', 'Brand'] = 'Atari'
df_completed.loc[df_completed.Platform == 'GEN', 'Brand'] = 'SEGA'
df_completed.loc[df_completed.Platform == 'GG', 'Brand'] = 'SEGA'
df_completed.loc[df_completed.Platform == 'SCD', 'Brand'] = 'SEGA'
df_completed.loc[df_completed.Platform == 'SAT', 'Brand'] = 'SEGA'
df_completed.loc[df_completed.Platform == 'DC', 'Brand'] = 'SEGA'
df_completed.loc[df_completed.Platform == 'NG', 'Brand'] = 'NEOGEO'
df_completed.loc[df_completed.Platform == '3DO', 'Brand'] = 'Panasonic'
df_completed.loc[df_completed.Platform == 'TG16', 'Brand'] = 'NEC'
df_completed.loc[df_completed.Platform == 'PCFX', 'Brand'] = 'NEC'
df_completed.loc[df_completed.Platform == 'WS', 'Brand'] = 'NEC'
df_completed.loc[df_completed.Platform == 'XB', 'Brand'] = 'Microsoft'
df_completed.loc[df_completed.Platform == 'X360', 'Brand'] = 'Microsoft'
df_completed.loc[df_completed.Platform == 'XOne', 'Brand'] = 'Microsoft'
df_completed.loc[df_completed.Platform == 'PC', 'Brand'] = 'PC'

# Observamos primeros registros
df_completed.head(5)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Brand
6896,6898,Checkers,2600,1980-01-01,Misc,Atari,0.22,0.01,0.0,0.00,0.24,Atari
2669,2671,Boxing,2600,1980-01-01,Fighting,Activision,0.72,0.04,0.0,0.01,0.77,Atari
5366,5368,Freeway,2600,1980-01-01,Action,Activision,0.32,0.02,0.0,0.00,0.34,Atari
1969,1971,Defender,2600,1980-01-01,Misc,Atari,0.99,0.05,0.0,0.01,1.05,Atari
1766,1768,Kaboom!,2600,1980-01-01,Misc,Activision,1.07,0.07,0.0,0.01,1.15,Atari


In [38]:
# Nos quedamos con las columnas necesarias
df_brand_war = df_completed.loc[:, ["Brand", "Year", "Global_Sales"]]

# Exportamos nuestro nuevo csv
df_brand_war.to_csv("Videogames/brand_war_consoles.csv",
                     encoding="utf-8")

Una vez tenemos la marca de la consola, vamos a realizar los mismos pasos que en el caso de la guerra por plataformas.

In [39]:
def process_csv_brand(filename):
    '''The function aims to obtain a dictionary of
    videoconsoles and their global videogame sales.
    
    The function has as input a .csv file containing the
    years, the name of the videoconsole and their videogame sales.
    Launch a dictionary separating by platforms, the years they
    have appeared and the total videogame sales in this year.
    
    Argument:
    - filename: csv file with data
    
    Return:
    - platforms: Dictionary with sorted data
    '''
    brands = {}
    dates = []
    
    with open(filename, 'rt', encoding='utf-8') as f:
        csv_reader = csv.DictReader(f)
        for row in csv_reader:
            brand = row['Brand']
            sales = row['Global_Sales']
            year = arrow.get(row['Year'], 'YYYY').format('YYYY')
            date = f'{year}'
            
            if brand not in brands:
                brands[brand] = {}
                    
            if date not in brands[brand]:
                brands[brand][date] = float(sales)
                    
            else:
                brands[brand][date] += float(sales)
                
            if date not in dates:
                dates.insert(0, date)
                    
    return brands

In [40]:
# Empleamos función anterior
df_brand_war = process_csv_brand('Videogames/brand_war_consoles.csv')

# Convertimos el diccionario en un dataframe
df_brand_war = pd.DataFrame(df_brand_war)

# Transponemos columnas por filas
df_brand_war = df_brand_war.transpose()

# Comprobamos orden de las columnas
df_brand_war.columns

Index(['1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2020', '2017'],
      dtype='object')

In [41]:
# Corregimos orden de las columnas
last_column = df_brand_war.pop('2020') 
  
# Insertamos de nuevo la columna
df_brand_war.insert(38, '2020', last_column)

# observamos de nuevo el resultado
df_brand_war.columns

Index(['1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2020'],
      dtype='object')

In [42]:
# Damos valores a los na
df_brand_war.fillna(0, inplace=True)

# Change data type from str to int
df_brand_war = df_brand_war[:].astype('float')

# Realizamos sumas acumuladas
for i in range(len(df_brand_war.columns)):
    if i == 0:
        pass
    else:
        df_brand_war[df_brand_war.columns[i]] = df_brand_war[
            df_brand_war.columns[i]] + df_brand_war[df_brand_war.columns[i-1]]
        
# Exportación del dataset listo
df_brand_war.to_csv("Videogames/brand_war_sales.csv",
                    encoding="utf-8")